## Part 1

In [1]:
import datetime
import re

In [2]:
with open("input.txt","r") as infile: 
    contents = [line for line in infile.read().split("\n") if len(line) > 0]
contents[0:10]

['[1518-10-31 00:58] wakes up',
 '[1518-02-27 00:57] wakes up',
 '[1518-04-05 00:03] falls asleep',
 '[1518-06-03 00:18] falls asleep',
 '[1518-08-06 00:39] falls asleep',
 '[1518-08-15 00:54] falls asleep',
 '[1518-03-07 00:00] falls asleep',
 '[1518-05-01 00:12] falls asleep',
 '[1518-06-17 00:53] wakes up',
 '[1518-03-13 00:13] falls asleep']

In [3]:
records = list()

for line in contents: 
    guard     = None
    timestamp = datetime.datetime.strptime(line[1:17], "%Y-%m-%d %H:%M")
    
    match = re.search('Guard (#\d+) begins shift', line.strip())
    if match:
        guard, action = match.group(1), "begins shift"
    else: 
        if "wakes up" in line: 
            action = "wakes up"
        elif "falls asleep" in line: 
            action = "falls asleep"
        else: 
            raise ValueError("Unrecognized input '{}'".format(line))
    
    records.append((timestamp, guard, action))

In [4]:
records.sort(key=lambda x: x[0])
records[0:2]

[(datetime.datetime(1518, 1, 29, 23, 57), '#2417', 'begins shift'),
 (datetime.datetime(1518, 1, 30, 0, 13), None, 'falls asleep')]

In [5]:
for i, record in enumerate(records):
    timestamp = record[0]

    if timestamp.hour >= 22: 
        timestamp = timestamp.replace(hour=0, minute=0) + datetime.timedelta(days=1)
    if timestamp.hour > 1: 
        timestamp = timestamp.replace(hour=1, minute=0)
        
    if record[1] is not None: 
        guard = record[1]
        
    records[i] = (timestamp, guard, record[2])

records[0:2]     

[(datetime.datetime(1518, 1, 30, 0, 0), '#2417', 'begins shift'),
 (datetime.datetime(1518, 1, 30, 0, 13), '#2417', 'falls asleep')]

In [6]:
#create a pivot table
#guard.date.records

pivot = dict()
for record in records: 
    if record[1] not in pivot: 
        pivot[record[1]] = dict()
    if record[0].date() not in pivot[record[1]]: 
        pivot[record[1]][record[0].date()] = list()
    pivot[record[1]][record[0].date()].append(record)

pivot["#2417"][datetime.date(1518,1,30)]

[(datetime.datetime(1518, 1, 30, 0, 0), '#2417', 'begins shift'),
 (datetime.datetime(1518, 1, 30, 0, 13), '#2417', 'falls asleep'),
 (datetime.datetime(1518, 1, 30, 0, 50), '#2417', 'wakes up')]

In [7]:
#sanity check
#check first record is begins shift
#check 'falls asleep' and 'wakes up' alternate in that order
for guard in pivot: 
    for date in pivot[guard]:
        if len(pivot[guard][date]) % 2 == 0: 
            raise ValueError("Expected uneven number of records for {}/{}".format(guard, date))
        for i, record in enumerate(pivot[guard][date]): 
            if i == 0: 
                if record[2] != "begins shift": 
                    raise ValueError("Expected 'begins shifts' instead of '{}'".format(record))
            elif i % 2 == 1: 
                if record[2] != 'falls asleep':
                    raise ValueError("Expected 'falls asleep' instead of '{}'".format(record))
            else: 
                if record[2] != 'wakes up':
                    raise ValueError("Expected 'wakes up' instead of '{}'".format(record))

In [8]:
#sum the minute of sleep by minute
for guard in pivot: 
    sleep = dict()
    for date in pivot[guard]: 
        for i in range(len(pivot[guard][date])): 
            if i % 2 == 1: 
                for minute in range(pivot[guard][date][i][0].minute, pivot[guard][date][i+1][0].minute):
                    if minute in sleep: 
                        sleep[minute] += 1
                    else:
                        sleep[minute] = 1
    pivot[guard]["sleep"] = sleep
    
sum(pivot["#2417"]["sleep"].values())

343

In [9]:
sleep, sleepiest = 0, None
for guard in pivot: 
    if sum(pivot[guard]["sleep"].values()) > sleep:
        sleep, sleepiest = sum(pivot[guard]["sleep"].values()), guard

"Guard {} was asleep for {} minutes in total".format(sleepiest, sleep)

'Guard #2663 was asleep for 522 minutes in total'

In [10]:
minutes, toughest = 0, None
for minute in pivot[sleepiest]["sleep"]: 
    if pivot[sleepiest]["sleep"][minute] > minutes: 
        minutes, toughest = pivot[sleepiest]["sleep"][minute], minute
        
"Guard {} was asleep the most often on minute {}".format(sleepiest, toughest)

'Guard #2663 was asleep the most often on minute 45'

In [11]:
"Solution: {}".format(int(sleepiest[1:]) * toughest)

'Solution: 119835'

## Part 2

In [12]:
#minute, length, guard
answer = 0, 0, None

for minute in range(0, 60): 
    for guard in pivot: 
        if minute in pivot[guard]["sleep"]:
            if pivot[guard]["sleep"][minute] > answer[1]: 
                answer = minute, pivot[guard]["sleep"][minute], guard

"Guard {} was asleep {} times on minute {}".format(answer[2], answer[1], answer[0])

'Guard #509 was asleep 20 times on minute 25'

In [13]:
"Solution: {}".format(answer[0] * int(answer[2][1:]))

'Solution: 12725'